In [11]:
# install necessary packages finta, gym, jupyterlab, matplotlib, mplfinance, numpy, pandas, quantstats, stable_baselines3, torch
# %pip install finta gym jupyterlab matplotlib mplfinance numpy pandas quantstats stable_baselines3 torch keras tensorflow


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import datetime
import mplfinance 
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from renkodf import Renko
from scipy.signal import lfilter

import sys
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/")
import fx # personal functions
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/")
from meta.env_fx_trading.env_fx import tgym
import finta

In [3]:
brick_size = 0.0003
# psar variables
start = 0.02
increment = 0.02
maximum = 0.2
# impulse variables
lengthMA = 34
lengthSignal = 9
# sma variables
sma_length = 3
smoothing_sma = 3

In [4]:
# read in the renko data
renko_full_data = pd.read_csv('C:/Users/WilliamFetzner/Documents/Trading/renko_full_data_1.csv')
# convert date to datetime
renko_full_data['datetime'] = pd.to_datetime(renko_full_data['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
renko_full_data = renko_full_data.set_index('datetime')
renko_full_data

,open,high,low,close,volume
datetime,,,,,
2022-11-15 02:00:54.735,1.0320,1.03230,1.03200,1.0323,98.0
2022-11-15 02:53:38.825,1.0320,1.03251,1.03170,1.0317,3867.0
2022-11-15 02:55:00.154,1.0317,1.03177,1.03140,1.0314,292.0
2022-11-15 03:00:04.310,1.0317,1.03200,1.03125,1.0320,497.0
2022-11-15 03:01:27.232,1.0320,1.03230,1.03195,1.0323,166.0
...,...,...,...,...,...
2024-03-06 21:14:54.215,1.0899,1.09008,1.08960,1.0896,1230.0
2024-03-06 22:23:07.131,1.0896,1.08988,1.08930,1.0893,3901.0
2024-03-06 22:58:31.495,1.0896,1.08990,1.08929,1.0899,1670.0


In [5]:
# calculate the sma of the open high low, close / 4 for 3 periods
renko_full_data['ohlc4'] = (renko_full_data['close'] + renko_full_data['open'] + renko_full_data['high'] + renko_full_data['low']) / 4
renko_full_data['sma'] = renko_full_data['ohlc4'].rolling(window=sma_length).mean()
# calculate the sma of sma3 for 3 periods
renko_full_data['smoothing_sma'] = renko_full_data['sma'].rolling(window=smoothing_sma).mean()
# find the difference between sma and smoothing_sma
renko_full_data['sma_diff'] = renko_full_data['sma'] - renko_full_data['smoothing_sma']
# find the sign of the sma_diff
renko_full_data['sma_sign'] = np.sign(renko_full_data['sma_diff'])
# find where the sma_sign changes from 1 to -1 or -1 to 1 or from 1
renko_full_data['sma_crossover'] = np.where((renko_full_data['sma_sign'] == 1) & (renko_full_data['sma_sign'].shift(1) == -1), 1, 
                                np.where((renko_full_data['sma_sign'] == -1) & (renko_full_data['sma_sign'].shift(1) == 1), -1, 
                                np.where((renko_full_data['sma_sign'] == -1) & (renko_full_data['sma_sign'].shift(1) == 0) & (renko_full_data['sma_sign'].shift(2) == 1), -1,
                                np.where((renko_full_data['sma_sign'] == 1) & (renko_full_data['sma_sign'].shift(1) == 0) & (renko_full_data['sma_sign'].shift(2) == -1), 1, 0))))
# when sma_crossover is 1 then buy, when sma_crossover is -1 then sell
renko_full_data['sma_signal'] = np.where(renko_full_data['sma_crossover'] == 1, 'buy',
                                np.where(renko_full_data['sma_crossover'] == -1, 'sell', 'nothing'))
# add a column for the width of bollinger bands
renko_full_data['bollinger_width'] = finta.TA.BBWIDTH(renko_full_data, period=20)
# add a column for the awesome oscillator
renko_full_data['awesome_oscillator'] = finta.TA.AO(renko_full_data)
# add the day of the week to the dataframe
renko_full_data['day_of_week'] = renko_full_data.index.day_name()
# place a 1 in day_of_week_transition, if it is the last bar on Friday and the next bar is Sunday
renko_full_data['day_of_week_transition'] = np.where((renko_full_data['day_of_week'] == 'Friday') & 
                                                     ((renko_full_data['day_of_week'].shift(-1) == 'Sunday') | 
                                                      (renko_full_data['day_of_week'].shift(-1) == 'Monday') |
                                                      (renko_full_data['day_of_week'].shift(-1) == 'Tuesday')), 1, 0)
renko_full_data.tail(10)

,open,high,low,close,volume,ohlc4,sma,smoothing_sma,sma_diff,sma_sign,sma_crossover,sma_signal,bollinger_width,awesome_oscillator,day_of_week,day_of_week_transition
datetime,,,,,,,,,,,,,,,,
2024-03-06 18:55:41.960,1.0911,1.09140,1.09110,1.0914,392.0,1.091250,1.090915,1.090688,0.000227,1.0,0,nothing,0.005022,0.002491,Wednesday,0
2024-03-06 19:11:58.650,1.0911,1.09153,1.09080,1.0908,2007.0,1.091057,1.091082,1.090881,0.000201,1.0,0,nothing,0.004847,0.002484,Wednesday,0
2024-03-06 19:34:35.013,1.0908,1.09124,1.09050,1.0905,1537.0,1.090760,1.091022,1.091006,0.000016,1.0,0,nothing,0.004293,0.002403,Wednesday,0
2024-03-06 19:42:04.739,1.0905,1.09058,1.09020,1.0902,717.0,1.090370,1.090729,1.090944,-0.000215,-1.0,-1,sell,0.003822,0.002261,Wednesday,0
2024-03-06 21:03:54.083,1.0902,1.09077,1.08990,1.0899,5141.0,1.090192,1.090441,1.090731,-0.000290,-1.0,0,nothing,0.003358,0.002025,Wednesday,0
2024-03-06 21:14:54.215,1.0899,1.09008,1.08960,1.0896,1230.0,1.089795,1.090119,1.090430,-0.000311,-1.0,0,nothing,0.002951,0.001640,Wednesday,0
2024-03-06 22:23:07.131,1.0896,1.08988,1.08930,1.0893,3901.0,1.089520,1.089836,1.090132,-0.000296,-1.0,0,nothing,0.002663,0.001227,Wednesday,0
2024-03-06 22:58:31.495,1.0896,1.08990,1.08929,1.0899,1670.0,1.089672,1.089662,1.089872,-0.000210,-1.0,0,nothing,0.002369,0.000888,Wednesday,0
2024-03-07 00:02:40.111,1.0896,1.08995,1.08930,1.0893,1579.0,1.089537,1.089577,1.089692,-0.000115,-1.0,0,nothing,0.002272,0.000653,Thursday,0


In [6]:
# add a column for the day of the week that is the day number 
renko_full_data['day'] = renko_full_data.index.dayofweek
renko_full_data_remove_cols = renko_full_data.drop(columns=['ohlc4', 'day_of_week'])
renko_full_data_remove_cols['symbol'] = 'EURUSD'

In [7]:
# resample the dataframe by week and put into a list called dfs
dfs = [group[1] for group in renko_full_data_remove_cols.groupby(renko_full_data_remove_cols.index.isocalendar().week)]
# reset the index for each dataframe in the list
for i in range(len(dfs)):
    dfs[i] = dfs[i].reset_index()
dfs[1]

,datetime,open,high,low,close,volume,sma,smoothing_sma,sma_diff,sma_sign,sma_crossover,sma_signal,bollinger_width,awesome_oscillator,day_of_week_transition,day,symbol
0,2023-01-09 00:00:00.195,1.0644,1.06480,1.06410,1.0641,8875.0,1.064335,1.064128,0.000207,1.0,0,nothing,0.003305,0.001958,0,0,EURUSD
1,2023-01-09 01:02:01.188,1.0644,1.06470,1.06381,1.0647,1247.0,1.064424,1.064317,0.000107,1.0,0,nothing,0.003264,0.002012,0,0,EURUSD
2,2023-01-09 01:19:40.152,1.0647,1.06500,1.06439,1.0650,1108.0,1.064508,1.064422,0.000086,1.0,0,nothing,0.003325,0.002053,0,0,EURUSD
3,2023-01-09 01:20:23.064,1.0650,1.06530,1.06500,1.0653,92.0,1.064775,1.064569,0.000206,1.0,0,nothing,0.003482,0.002155,0,0,EURUSD
4,2023-01-09 01:29:02.305,1.0653,1.06560,1.06512,1.0656,929.0,1.065109,1.064797,0.000312,1.0,0,nothing,0.003412,0.002199,0,0,EURUSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719,2024-01-12 20:31:34.448,1.0956,1.09590,1.09540,1.0959,1113.0,1.095407,1.095413,-0.000006,-1.0,0,nothing,0.003925,-0.001372,0,4,EURUSD
1720,2024-01-12 21:12:40.675,1.0959,1.09620,1.09552,1.0962,1669.0,1.095652,1.095464,0.000188,1.0,1,buy,0.003828,-0.001359,0,4,EURUSD
1721,2024-01-12 21:41:54.536,1.0959,1.09633,1.09560,1.0956,1295.0,1.095837,1.095632,0.000206,1.0,0,nothing,0.003897,-0.001272,0,4,EURUSD
1722,2024-01-12 22:15:46.725,1.0956,1.09594,1.09530,1.0953,1485.0,1.095782,1.095757,0.000025,1.0,0,nothing,0.003872,-0.001207,0,4,EURUSD


In [8]:
renko_full_data_remove_cols.columns

Index(['open', 'high', 'low', 'close', 'volume', 'sma', 'smoothing_sma',
       'sma_diff', 'sma_sign', 'sma_crossover', 'sma_signal',
       'bollinger_width', 'awesome_oscillator', 'day_of_week_transition',
       'day', 'symbol'],
      dtype='object')

In [9]:
num_weeks = len(dfs)
weekly_train = dfs[0:int(num_weeks*0.8)]

In [10]:
# save all of the dataframes to csv files
files = []
for i in range(len(weekly_train)):
    path = 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week' + str(i) + '.csv'
    # weekly_train[i].to_csv(path, index=False)
    files.append(path)
files_train = files[:10]
files_train

['C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week0.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week1.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week2.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week3.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week4.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week5.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week6.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week7.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/renko_train_week8.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_

# Reinforcement Learning

In [11]:
# def train(env, agent, files,if_vix = True,**kwargs):
#     learning_rate = kwargs.get('learning_rate', 2 ** -15)
#     batch_size = kwargs.get('batch_size', 2 ** 11 )
#     gamma = kwargs.get('gamma', 0.99)
#     seed = kwargs.get('seed', 312)
#     total_timesteps = kwargs.get('total_timesteps', 1e6)
#     net_dimension = kwargs.get('net_dimension', 2**9)
#     cwd = kwargs.get('cwd','./'+str(agent))

#     # env_instance = map(env, [pd.read_csv(f) for f in files])
#     if agent == 'ppo':
#         from stable_baselines3 import PPO
#         from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv

#         # env_train = [x.get_sb_env for x in env_instance ]
#         vector_env = [lambda:env(df=pd.read_csv(f)) for f in files]
#         env_train = SubprocVecEnv(vector_env)
#         model = PPO("MlpPolicy", env_train, learning_rate=learning_rate, 
#                     n_steps=2048, batch_size=batch_size, ent_coef=0.0, 
#                     gamma=gamma, seed=seed)
#         start_time = time.time()
#         s = datetime.datetime.now()
#         print(f'Training start: {s}')
#         model.learn(total_timesteps=total_timesteps, tb_log_name = 'ppo')
#         print('Training finished!')
#         model_name = "./data/models/EURUSD_renko_week_" + s.strftime('%Y%m%d%H%M%S')
#         model.save(model_name)
#         print(f'Trained model saved in {model_name}')
#         print(f"training time: {(time.time() - start_time)}")

#     else:
#         raise ValueError('DRL library input is NOT supported. Please check.')
# train(env=tgym, agent="ppo", files=files_train)

## Keras Architecture
In this code snippet:

- We import necessary modules from TensorFlow and Keras.

- We define functions build_actor_network and build_critic_network to create the actor and critic networks using Keras' functional API.

- The actor network outputs a probability distribution over actions using softmax activation.

- The critic network estimates the state value function.

In [13]:
# Define the Keras model architecture for the actor network
def build_actor_network(state_dim, action_dim):
    inputs = layers.Input(shape=(state_dim,))
    dense1 = layers.Dense(64, activation='relu')(inputs)
    dense2 = layers.Dense(64, activation='relu')(dense1)
    outputs = layers.Dense(action_dim, activation='softmax')(dense2)  # Softmax for policy output
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Define the Keras model architecture for the critic network
def build_critic_network(state_dim):
    inputs = layers.Input(shape=(state_dim,))
    dense1 = layers.Dense(64, activation='relu')(inputs)
    dense2 = layers.Dense(64, activation='relu')(dense1)
    outputs = layers.Dense(1)(dense2)  # Linear output for value estimation
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage:
state_dim = 4  # Example state dimension
action_dim = 2  # Example action dimension

actor_model = build_actor_network(state_dim, action_dim)
critic_model = build_critic_network(state_dim)

actor_model.summary()
critic_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,610 (18.01 KB)

 Trainable params: 4,610 (18.01 KB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,545 (17.75 KB)

 Trainable params: 4,545 (17.75 KB)

 Non-trainable params: 0 (0.00 B)

# PPO Loss Function
In this code snippet:

- The ppo_loss function defines a custom loss function that calculates the PPO loss based on the clipped surrogate objective.
- The loss function computes the PPO loss using the ratio of new probabilities to old probabilities and applies clipping to ensure stable updates.
- An entropy term is included in the loss function to encourage exploration.
- Finally, the actor model is compiled using the PPO loss function for training.


In [ ]:
# PPO loss function
def ppo_loss(old_probs, advantages, clip_ratio):
    def loss(y_true, y_pred):
        new_probs = y_pred
        ratio = new_probs / old_probs

        clipped_ratio = tf.clip_by_value(ratio, 1 - clip_ratio, 1 + clip_ratio)
        surrogate_objective = tf.minimum(ratio * advantages, clipped_ratio * advantages)

        entropy = -tf.reduce_sum(new_probs * tf.math.log(new_probs + 1e-10), axis=-1)  # Entropy bonus

        ppo_loss = -tf.reduce_mean(surrogate_objective - 0.01 * entropy)  # Include entropy for exploration

        return ppo_loss

    return loss

# Example usage:
clip_ratio = 0.2  # Example clip ratio
advantages = np.random.rand(10)  # Example advantages array
old_probs = np.random.rand(10, 2)  # Example old probabilities array

# Compile the actor model with PPO loss function
actor_model.compile(optimizer=keras.optimizers.Adam(), loss=ppo_loss(old_probs, advantages, clip_ratio))

# Training Loop

In this code snippet:
- The train_ppo function defines a training loop that collects experiences from interacting with the environment and performs PPO updates using the collected data.
- The compute_advantages function calculates advantages using rewards and values from the critic network.
- The PPO agent is initialized with actor and critic models.
- The training loop is executed for a specified number of epochs on an example environment (CartPole-v1).

In [ ]:
# Define the training loop for PPO agent
def train_ppo(agent, env, epochs, batch_size):
    for epoch in range(epochs):
        states = []
        actions = []
        rewards = []
        old_probs = []
        dones = []

        state = env.reset()
        done = False

        while not done:
            action_probs = agent.actor_model.predict(np.array([state]))[0]
            action = np.random.choice(len(action_probs), p=action_probs)
            next_state, reward, done, _ = env.step(action)

            states.append(state)
            actions.append(action)
            rewards.append(reward)
            old_probs.append(action_probs)
            dones.append(done)

            state = next_state

        # Compute advantages using rewards and values from critic network
        advantages = compute_advantages(rewards, dones, agent.critic_model)

        # Perform PPO update using collected data
        actor_loss = agent.actor_model.train_on_batch(np.array(states), np.array(actions), sample_weight=advantages)
        critic_loss = agent.critic_model.train_on_batch(np.array(states), np.array(rewards))

        print(f"Epoch: {epoch + 1}, Actor Loss: {actor_loss}, Critic Loss: {critic_loss}")

# Example usage:
class PPOAgent:
    def __init__(self, actor_model, critic_model):
        self.actor_model = actor_model
        self.critic_model = critic_model

env = gym.make('CartPole-v1')  # Example environment
actor_model = build_actor_network(env.observation_space.shape[0], env.action_space.n)
critic_model = build_critic_network(env.observation_space.shape[0])
agent = PPOAgent(actor_model, critic_model)

train_ppo(agent, env, epochs=100, batch_size=32)